In [ ]:
! pip install chardet



In [ ]:
import os
# import gzip
# import chardet

class FileValidator:
    def __init__(self, max_size_mb=5):
        """Initialize validator with size limit (default 5MB)."""
        self.max_size = max_size_mb * 1024 * 1024

    def validate(self, file_path):
        """Validate file step by step."""
        results = {
            "status": "PASS",
            "reason": None,
            "error_path": None
        }

        # Step 1: Extension check
        if not self._is_valid_extension(file_path):
            results["status"] = "FAIL"
            results["reason"] = "NOT_JSONL"
            results["error_path"] = "/error/not_jsonl/"
            return results

        # # Step 2: Encoding check (UTF-8)
        # if not self._is_valid_utf8(file_path):
        #     results["status"] = "FAIL"
        #     results["reason"] = "BAD_ENCODING"
        #     results["error_path"] = "/error/bad_encoding/"
        #     return results

        # Step 3: Size and empty check
        if not self._is_valid_size(file_path):
            results["status"] = "FAIL"
            results["reason"] = "EMPTY_FILE or TOO_LARGE"
            results["error_path"] = "/error/empty_file/"  # or /error/too_large/
            return results

        return results

    def _is_valid_extension(self, file_path):
        allowed_ext = [".jsonl", ".ndjson", ".jsonl.gz", ".ndjson.gz",".json"]
        return any(file_path.endswith(ext) for ext in allowed_ext)

    # def _is_valid_utf8(self, file_path):
    #     try:
    #         # Handle gzipped files
    #         open_func = gzip.open if file_path.endswith(".gz") else open
    #         with open_func(file_path, 'rb') as f:
    #             raw_data = f.read(4096)
    #             result = chardet.detect(raw_data)
    #             return result['encoding'] == 'utf-8'
    #     except Exception as e:
    #         print(f"Encoding check error: {e}")
    #         return False

    def _is_valid_size(self, file_path):
        size = os.path.getsize(file_path)
        if size == 0:
            print("File is empty")
            return False
        if size > self.max_size:
            print(f"File too large: {size} bytes")
            return False
        return True


In [4]:
validator = FileValidator(max_size_mb=10)

file_path = r"D:\AI Projects\powerthon\powerthon_AI\external_data\new_output_jsons\new_output_jsons\api_raw_json_analytics_20251017\861850060226525_20251017161523.json"  # replace with your test file
result = validator.validate(file_path)
print(result)


{'status': 'PASS', 'reason': None, 'error_path': None}


convert ascii into utf-8

hi


In [ ]:
import chardet

# 🔹 Step 1: Give your JSON file path here
file_path = r"D:\AI Projects\powerthon\powerthon_AI\external_data\new_output_jsons\new_output_jsons\api_raw_json_analytics_20251017\861850060226657_20251017180005_utf8.json"   # ← replace with your file path

# 🔹 Step 2: Read file in binary mode
with open(file_path, 'rb') as f:
    raw_data = f.read()

# 🔹 Step 3: Detect encoding
result = chardet.detect(raw_data)
detected_encoding = result['encoding']
confidence = result['confidence']

print(f"📘 Detected Encoding: {detected_encoding}")
print(f"🔍 Confidence: {confidence*100:.2f}%")

# 🔹 Step 4: Check UTF-8 status
if detected_encoding and detected_encoding.lower() == 'utf-8':
    print("✅ The file is UTF-8 encoded.")
else:
    print("⚠️ The file is NOT UTF-8 encoded.")


In [ ]:
import json

# 🔹 Give the same file path
file_path = r"D:\AI Projects\powerthon\powerthon_AI\external_data\new_output_jsons\new_output_jsons\api_raw_json_analytics_20251017\861850060226657_20251017180005.json"   # ← same file as above

# 🔹 Read and detect encoding again
with open(file_path, 'rb') as f:
    raw_data = f.read()

import chardet
detected_encoding = chardet.detect(raw_data)['encoding']

# 🔹 Convert and save as UTF-8
try:
    text = raw_data.decode(detected_encoding)
    data = json.loads(text)  # validate JSON format

    new_path = file_path.replace(".json", "_utf8.json")
    with open(new_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"✅ File successfully converted and saved as: {new_path}")

except Exception as e:
    print(f"❌ Conversion failed: {e}")


one by one adding 

In [ ]:
import os
import json
   

class FileValidator:
    def __init__(self, max_size_mb=5):
        """Initialize validator with size limit (default 5MB)."""
        self.max_size = max_size_mb * 1024 * 1024

    def validate(self, file_path):
        """Validate file step by step."""
        results = {
            "status": "PASS",
            "reason": None,
            "error_path": None,
            "skipped_lines": 0
        }

        # Step 1: Extension check
        if not self._is_valid_extension(file_path):
            results["status"] = "FAIL"
            results["reason"] = "NOT_JSONL"
            results["error_path"] = "/error/not_jsonl/"
            return results

        # Step 2: (Optional) Encoding check
        # if not self._is_valid_utf8(file_path):
        #     results["status"] = "FAIL"
        #     results["reason"] = "BAD_ENCODING"
        #     results["error_path"] = "/error/bad_encoding/"
        #     return results

        # Step 3: Size and empty check
        if not self._is_valid_size(file_path):
            results["status"] = "FAIL"
            results["reason"] = "EMPTY_FILE or TOO_LARGE"
            results["error_path"] = "/error/empty_file/"
            return results

        # Step 4: Record-level validation (skip blank lines, check JSON)
        record_check = self._validate_records(file_path)
        if record_check["status"] == "FAIL":
            return record_check  # return early if JSON parse fails

        results["skipped_lines"] = record_check["skipped_lines"]
        return results

    # -----------------------
    # Helper Functions
    # -----------------------

    def _is_valid_extension(self, file_path):
        allowed_ext = [".jsonl", ".ndjson", ".jsonl.gz", ".ndjson.gz", ".json"]
        return any(file_path.endswith(ext) for ext in allowed_ext)

    # def _is_valid_utf8(self, file_path):
    #     try:
    #         open_func = gzip.open if file_path.endswith(".gz") else open
    #         with open_func(file_path, 'rb') as f:
    #             raw_data = f.read(4096)
    #             result = chardet.detect(raw_data)
    #             return result['encoding'] == 'utf-8'
    #     except Exception as e:
    #         print(f"Encoding check error: {e}")
    #         return False

    def _is_valid_size(self, file_path):
        size = os.path.getsize(file_path)
        if size == 0:
            print("File is empty")
            return False
        if size > self.max_size:
            print(f"File too large: {size} bytes")
            return False
        return True

    def _validate_records(self, file_path):
        """Skip blank lines and validate JSON structure line by line."""
        results = {
            "status": "PASS",
            "reason": None,
            "error_path": None,
            "skipped_lines": 0
        }

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                for line_num, line in enumerate(f, start=1):
                    stripped = line.strip()

                    # Step 6: Skip blank or whitespace-only lines
                    if not stripped:
                        results["skipped_lines"] += 1
                        continue

                    # Step 7: Strict JSON parsing
                    # try:
                    #     json.loads(stripped)
                    # except json.JSONDecodeError as e:
                    #     print(f"Line {line_num} JSON parse error: {e}")
                    #     results["status"] = "FAIL"
                    #     results["reason"] = "LINE_PARSE"
                    #     results["error_path"] = "/error/line_parse/"
                    #     return results  # stop checking further
        except Exception as e:
            print(f"Record validation error: {e}")
            results["status"] = "FAIL"
            results["reason"] = "RECORD_VALIDATION_ERROR"
            results["error_path"] = "/error/record_validation/"
            return results

        return results


In [13]:
validator = FileValidator(max_size_mb=10)

file_path = r"D:\AI Projects\powerthon\powerthon_AI\external_data\new_output_jsons\new_output_jsons\api_raw_json_analytics_20251017\861850060226525_20251017163031.json"  # replace with your test file
result = validator.validate(file_path)
print(result)


{'status': 'PASS', 'reason': None, 'error_path': None, 'skipped_lines': 0}


#new one ny one adding 

In [ ]:
import os
import json
from datetime import datetime
from dateutil import parser as date_parser
# import gzip
# import chardet

class FileValidator:
    def __init__(self, max_size_mb=5):
        """Initialize validator with size limit (default 5MB)."""
        self.max_size = max_size_mb * 1024 * 1024

    def validate(self, file_path):
        """Validate file step by step."""
        results = {
            "status": "PASS",
            "reason": None,
            "error_path": None,
            "skipped_lines": 0
        }

        # Step 1: Extension check
        if not self._is_valid_extension(file_path):
            results["status"] = "FAIL"
            results["reason"] = "NOT_JSONL"
            results["error_path"] = "/error/not_jsonl/"
            return results

        # Step 2: (Optional) Encoding check (disabled)
        # if not self._is_valid_utf8(file_path):
        #     results["status"] = "FAIL"
        #     results["reason"] = "BAD_ENCODING"
        #     results["error_path"] = "/error/bad_encoding/"
        #     return results

        # Step 3: Size and empty check
        if not self._is_valid_size(file_path):
            results["status"] = "FAIL"
            results["reason"] = "EMPTY_FILE or TOO_LARGE"
            results["error_path"] = "/error/empty_file/"
            return results

        # Step 4–10: Record-level validation
        record_check = self._validate_records(file_path)
        if record_check["status"] == "FAIL":
            return record_check

        results["skipped_lines"] = record_check["skipped_lines"]
        return results

    # -----------------------
    # Helper Functions
    # -----------------------

    def _is_valid_extension(self, file_path):
        allowed_ext = [".jsonl", ".ndjson", ".jsonl.gz", ".ndjson.gz", ".json"]
        return any(file_path.endswith(ext) for ext in allowed_ext)
    
    # def _is_valid_utf8(self, file_path):
    #     try:
    #         open_func = gzip.open if file_path.endswith(".gz") else open
    #         with open_func(file_path, 'rb') as f:
    #             raw_data = f.read(4096)
    #             result = chardet.detect(raw_data)
    #             return result['encoding'] == 'utf-8'
    #     except Exception as e:
    #         print(f"Encoding check error: {e}")
    #         return False

    def _is_valid_size(self, file_path):
        size = os.path.getsize(file_path)
        if size == 0:
            print("File is empty")
            return False
        if size > self.max_size:
            print(f"File too large: {size} bytes")
            return False
        return True

    # -----------------------
    # Record-Level Validation
    # -----------------------

    def _validate_records(self, file_path):
        """Skip blank lines, check JSON validity, type coercion, and timestamp normalization."""
        results = {
            "status": "PASS",
            "reason": None,
            "error_path": None,
            "skipped_lines": 0
        }

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                for line_num, line in enumerate(f, start=1):
                    stripped = line.strip()

                    # Step 6: Skip blank lines
                    if not stripped:
                        results["skipped_lines"] += 1
                        continue

                    # Step 7: JSON structure validation
                    try:
                        record = json.loads(stripped)
                    except json.JSONDecodeError as e:
                        print(f"Line {line_num} JSON parse error: {e}")
                        results["status"] = "FAIL"
                        results["reason"] = "LINE_PARSE"
                        results["error_path"] = "/error/line_parse/"
                        return results

                    # Step 9: Type coercion check (numeric fields)
                    if not self._check_type_coercion(record, line_num, results):
                        return results

                    # Step 10: Timestamp normalization & validation
                    if not self._check_timestamps(record, line_num, results):
                        return results

        except Exception as e:
            print(f"Record validation error: {e}")
            results["status"] = "FAIL"
            results["reason"] = "RECORD_VALIDATION_ERROR"
            results["error_path"] = "/error/record_validation/"
            return results

        return results

    # -----------------------
    # Step 9 – Type Coercion
    # -----------------------

    def _check_type_coercion(self, record, line_num, results):
        """Try coercing numeric-like values to int/float. If fails → DQ failed."""
        try:
            for key, value in record.items():
                # Skip non-string types
                if not isinstance(value, str):
                    continue

                # Check if string looks numeric
                if value.replace(".", "", 1).isdigit():
                    try:
                        # Try float or int conversion
                        _ = float(value) if "." in value else int(value)
                    except Exception:
                        print(f"Line {line_num} DQ failed - cannot coerce {key}='{value}'")
                        results["status"] = "FAIL"
                        results["reason"] = "DQ_FAILED"
                        results["error_path"] = "/error/dq_failed/"
                        return False
        except Exception as e:
            print(f"Type coercion error on line {line_num}: {e}")
            results["status"] = "FAIL"
            results["reason"] = "DQ_FAILED"
            results["error_path"] = "/error/dq_failed/"
            return False
        return True

    # -----------------------
    # Step 10 – Timestamp Check
    # -----------------------

    def _check_timestamps(self, record, line_num, results):
        """Check if timestamp fields can be parsed & normalized to UTC ISO-8601."""
        try:
            for key, value in record.items():
                if not isinstance(value, str):
                    continue

                # Check keys that look like timestamps or ISO strings
                if any(term in key.lower() for term in ["time", "date", "timestamp","ReadingTimeStamp","stage_timestamp","alidation_timestamp","processed_at"]):
                    try:
                        parsed_time = date_parser.parse(value)
                        _ = parsed_time.astimezone().isoformat()  # convert to UTC ISO-8601
                    except Exception:
                        print(f"Line {line_num} invalid timestamp for {key}: {value}")
                        results["status"] = "FAIL"
                        results["reason"] = "INVALID_TIMESTAMP"
                        results["error_path"] = "/error/timestamp_invalid/"
                        return False
        except Exception as e:
            print(f"Timestamp normalization error on line {line_num}: {e}")
            results["status"] = "FAIL"
            results["reason"] = "INVALID_TIMESTAMP"
            results["error_path"] = "/error/timestamp_invalid/"
            return False
        return True


NEW VALIDATIONS
